In [22]:
import numpy as np
import pandas as pd
from trax import fastmath
from trax import layers as tl
from trax import shapes
from trax.fastmath import numpy as jnp  # For use in defining new layer types.
from trax.shapes import ShapeDtype
from trax.shapes import signature
from trax import supervised as ts
from trax.supervised import training
import trax
from trax.fastmath import jax
from trax.fastmath import jit
import random as rnd
import trax.data as td

In [23]:
td_imdb = td.TFDS('imdb_reviews', keys=('text', 'label'), train=True)

/home/pd/NNBasics/venv_NNBasics/bin/python


In [42]:
inputs = td.Serial(
  td.TFDS('imdb_reviews', keys=('text', 'label'), train=True),
  td.Tokenize(vocab_file='en_8k.subword', keys=[0]),
  td.Shuffle(),
  td.FilterByLength(max_length=2048, length_keys=[0]),
  td.BucketByLength(boundaries=[  32, 128, 512, 2048],
                      batch_sizes=[128,  32,   8,    2, 1],
                      length_keys=[0]),
  td.AddLossWeights()
)

# inputs_test = td.Serial(
#   td.TFDS('imdb_reviews', keys=('text', 'label'), train=False),
#   td.Tokenize(vocab_file='en_8k.subword', keys=[0]),
#   td.Shuffle(),
#   td.FilterByLength(max_length=2048, length_keys=[0]),
#   td.BucketByLength(boundaries=[  32, 128, 512, 2048],
#                       batch_sizes=[128,  32,   8,    2, 1],
#                       length_keys=[0]),
#   td.AddLossWeights()
# )

In [77]:
def get_tasks(gen):

    tt = training.TrainTask(
        labeled_data=inputs(),
        loss_layer = tl.WeightedCategoryCrossEntropy(),
        optimizer=trax.optimizers.Adam(0.01),
        n_steps_per_checkpoint=5,
    )

    et = training.EvalTask(
        labeled_data=inputs(),        
        metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()],
    )

    return tt,et

DenseSentiment = tl.Serial(
    tl.Embedding(vocab_size=8000,d_feature=256),
    tl.Mean(axis=1),
    tl.Dense(n_units = 2),
    tl.LogSoftmax()
)

tt,et = get_tasks(inputs())

training_loop = training.Loop( 
                            DenseSentiment, # The learning model
                            tt, # The training task
                            eval_tasks= et, # The evaluation task
) 

Will not write evaluation metrics, because output_dir is None.


2022-12-15 11:44:57.750446: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-12-15 11:44:57.770026: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [94]:
training_loop.run(n_steps = 2)
#training_loop.history.get('train','metrics/WeightedCategoryCrossEntropy')
training_loop.history.get('eval','metrics/WeightedCategoryAccuracy')

Did not save checkpoint as output_dir is None

Step     30: Ran 2 train steps in 0.07 secs
Step     30: train WeightedCategoryCrossEntropy |  0.68922001
Step     30: eval  WeightedCategoryCrossEntropy |  0.71968335
Step     30: eval      WeightedCategoryAccuracy |  0.37500000


[(1, 0.375),
 (5, 0.5),
 (10, 0.5),
 (15, 1.0),
 (20, 0.5),
 (25, 0.625),
 (30, 0.375)]

In [89]:
help(trax.supervised.history.History)

Help on class History in module trax.supervised.history:

class History(builtins.object)
 |  History of metrics.
 |  
 |  History contains the metrics recorded during training and evaluation.
 |  Save data with history.append and get a sequence of data by calling
 |  history.get.
 |  
 |  For example:
 |  history.append('train', 'metrics/accuracy', 1, 0.04)
 |  history.append('train', 'metrics/accuracy', 1000, 0.31)
 |  history.get('train', 'metrics/accuracy')
 |  # returns [(1, 0.04), (1000, 0.31)]
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  append(self, mode, metric, step, value)
 |      Append (step, value) pair to history for the given mode and metric.
 |  
 |  get(self, mode, metric)
 |      Get the history for the given metric and mode.
 |  
 |  metrics_for_mode(self, mode)
 |      Metrics available for a given mode.
 |  
 |  to_dict(self)
 |

In [38]:
training_loop.load_checkpoint(directory='out1')

TypeError: unsupported operand type(s) for ==: 'DeviceArray' and 'tuple'

In [52]:
import pickle,gzip
with gzip.open('out1/model_90.pkl.gz','rb') as f:
    dat = pickle.load(f)
    dir(dat)

In [59]:
pd.DataFrame(dat.items())

,0,1
0,step,90
1,flat_weights,2
2,flat_state,[]
3,flat_eval_state,[]
4,history,{'_values': {'train': {'metrics/WeightedCatego...
5,slots_per_task,2
6,input_signature,"(ShapeDtype{shape:(2, 1024), dtype:int64},)"
7,version_timestamp,Mar-10-2021


In [57]:
help(training_loop.run)

Help on method run in module trax.supervised.training:

run(n_steps=1) method of trax.supervised.training.Loop instance
    Runs this training loop for n steps.
    
    Optionally runs evals and saves checkpoints at specified points.
    
    Args:
      n_steps: Stop training after completing n steps.

